# Ansatzes

The present notebook explains how to use the different functions from the Python module *ansatzes* of the package **PH.ansatzes**. These functions are used for implementing and solving easily different typical ansatzes that are used in the **VQE**

In [ ]:
import logging
logging.basicConfig(
    format='%(asctime)s-%(levelname)s: %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.INFO
    #level=logging.DEBUG
)
logger = logging.getLogger('__name__')

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import sys
sys.path.append("../../")

## 0. The QPU

To simulate the quantum circuits generated by the functions presented in this notebook a configured myQLM (or QLM) **Quantum Process Unit (QPU)** is mandatory. The **QPU** can execute an ideal simulation or can simulate the quantum circuits under a noisy hardware model (noisy simulation). To easily deal with these 2 kinds of simulations the  *select_qpu* function from **PH.qpu.select_qpu** was developed. The input of this function is a Python dictionary that allows to the user configure easily a **QPU**.

In the present notebook, only ideal simulation is used. Please refer to the **PH/qpu/NoisyModels.ipynb** notebook for configuring noisy models and the corresponding noisy **QPU**s

**BE AWARE**

For the ansatz computation mandatory for getting the corresponding **Parent Hamiltonian** the exact complete state is mandatory so for these computations ideal simulation (i.e. ideal qpus) **SHOULD BE USED**

In [ ]:
# myQLM qpus
from PH.qpu.select_qpu import select_qpu

The minimum Python dictionary for configuring an ideal **QPU** is presented in the following cell. In this case, the user only has to provide a value to the *qpu_type* key. Depending on the type of simulator desired the following strings should be provided:

* *qlmass_linalg*: to use the **LinAlg Quantum Learning Machine (QLM)** algebra simulator. In this case, the computation will be sent to the **QLM** by using the  Qaptiva QLM as a Service.
* *qlmass_mps*: to use **MPS QLM** simulator. In this case, the computation will be sent to the **QLM** by using the  Qaptiva QLM as a Service.
* *python*: to use the PyLinalg algebra simulator.
* *c*: to use the CLinalg alegbra simulator.
* *linalg*: to use the **LinAlg QLM**. In this case, the user should be inside a **EVIDEN QLM**
* *mps*: to use the **MPS QLM** simulator. In this case, the user should be inside a **EVIDEN QLM**

In [ ]:
# List with the strings that should be provided for an ideal QPU
ideal_qpus = ["c", "python", "linalg", "mps", "qlmass_linalg", "qlmass_mps"]
qpu_config = {
    "qpu_type": ideal_qpus[0], 
}
qpu = select_qpu(qpu_config)

## 1. ansatzes module

One mandatory step for using the Parent Hamiltonian, **PH**, library (see notebook **03_Using_PH_Class.ipynb**) is computing for a given ansatz its complete state. This is the amplitudes of the state in the computational $n$ qubit basis. 

In the *ansatzes* module of the library package **PH.ansatzes** several functions and classes for dealing with this part of the computation were done.


### 1.1 SolveCircuit class

The **SolveCircuit** takes a *Atos myqlm circuit* with an ansatz, fixes their parameter, simulates using *Atos qpu* and returns the state of the ansatz.

The main input of this class is a QLM circuit where the parameters were set. For showing how this class works we are going to use an ansatz example: the translational invariant circuit of the original Parent Hamiltonian paper.

In [ ]:
from PH.ansatzes.ansatzes import SolveCircuit

#### 1.1.1 Parent Hamiltonian Github ansatz

The *ansatz_qlm_01* function, from **PH.ansatzes.ansatzes** module, implements a **Atos myqlm** version of the ansatz in the github:

https://github.com/FumiKobayashi/Parent_Hamiltonian_as_a_benchmark_problem_for_variational_quantum_eigensolvers

from the original Parent Hamiltonian Papper:

* Kobayashi, F., Mitarai, K., & Fujii, K. (2022). Parent Hamiltonian as a benchmark problem for variational quantum eigensolvers. Phys. Rev. A, 105, 052415 (https://doi.org/10.1103%2Fphysreva.105.052415)



In [ ]:
from PH.ansatzes.ansatzes import ansatz_qlm_01

We need to provided to the *ansatz_qlm_01* the number of desired qubits and the depth of the circuit. The function returns an *Atos myqlm circuit* of the ansatz.

In [ ]:
n_qubits = 12
depth = 2
circ_ansatz01 = ansatz_qlm_01(nqubits=n_qubits, depth=depth)

In [ ]:
%qatdisplay circ_ansatz01 --svg

#### 1.1.2 . Setting the parameters

As can be seen, the circuit has the parameters as variables. We can set the parameters by calling the *angles_ansatz01* function. If $n_l$ is the number of layers of the circuit the formula for setting the parameters is:


$$\delta \theta = \frac{\pi}{4*(n_l+1)}$$

$$\theta_i = (i+1) \delta \theta \; i=0, 1, \cdots 2n_l-1$$

The outputs of the function are:
* circuit with the parameters fixed
* pandas DataFrame with the value of the parameters used

In [ ]:
from PH.ansatzes.ansatzes import angles_ansatz01

In [ ]:
circ_ansatz01_, pdf_par = angles_ansatz01(circ_ansatz01)

In [ ]:
# Now the vatiables are fixed to values
%qatdisplay circ_ansatz01_ --svg

In [ ]:
# pandas DataFrame with parameters of the circuit
pdf_par

In addition, we can use the parameters of the circuit providing a pandas DataFrame to the function. In this case, the function can be used for setting the parameters for any circuit! The parameters should be passed as a pandas DataFrame with 2 columns:

* key: string with the name of the parameter (the same name that is in the circuit)
* value: float value of the correspondent parameter

In [ ]:
# Random parameters
parameters = {v_ : 2 * np.pi * np.random.rand() for i_, v_ in enumerate(circ_ansatz01.get_variables())}
# Creating the DataFrame
angles = [k for k, v in parameters.items()]
values = [v for k, v in parameters.items()]
pdf_parameters = pd.DataFrame([angles, values], index=['key', 'value']).T

In [ ]:
pdf_parameters

In [ ]:
circ_ansatz01_2, _ = angles_ansatz01(circ_ansatz01, pdf_parameters)

In [ ]:
# The output is the circuit with the parameters fixed
%qatdisplay circ_ansatz01_2 --svg

#### 1.1.3 Solving the ansatz

Now we can use the *SolveCircuit* class to solve the circuit. The main inputs are:
* circuit: QLM circuit where the parameters were set
* Input dictionary with the following keys:
    * nqubits: number of qubits of the ansatz
    * qpu: myqlm QPU used for solving the circuit
    * parameters: a pandas DataFrame with the parameters of the circuit
    * save: For saving or not the parameters and the solution (the state) of the circuit
    * filename: complete base filename for storing the parameters and state

For solving the circuit the *run* method of the object should be invoked

In [ ]:
from PH.utils.utils_ph import create_folder

filename = "ansatz_{}_depth_{}_nqubits_{}".format("simple01", depth, n_qubits)
folder = create_folder("Savings_notebook1")

class_dict = {
    'nqubits' : n_qubits,
    "qpu" : qpu,
    "parameters" : pdf_parameters,
    "filename": folder + filename,
    "save": True
}

In [ ]:
solv_ansatz01 = SolveCircuit(circ_ansatz01_, **class_dict)

In [ ]:
solv_ansatz01.run()

For simulating the ansatz the *run* method of the class should be executed when proper configuration is done. The *state* attribute will contain a pandas DataFrame with complete state information

In [ ]:
solv_ansatz01.state

In addition, three files were created (if requested):

* folder + filename*_parameters.csv*: where the parameters were stored
* folder + filename*_state.csv*: where the state of the ansat was stored. Only the Amplitudes will be stored.
* folder + filename*_ansatz_time.csv*: where the time of solving the ansatz was stored

### 1.2 Parent Hamiltonian General ansatz

Other ansatzes are implemented in the **PH.ansatzes.ansatzes ** module.

The function *ansatz_qlm_02* implements a generalization of the *ansatz_qlm_01* one. In the *ansatz_qlm_01* all the qubits have the same operations with the same parameters. In the *ansatz_qlm_02* each qubit has the same operations but each operation will have a different parameter.

The *SolveCircuit* class can be used for solving the ansatz

In [ ]:
from PH.ansatzes.ansatzes import ansatz_qlm_02

In [ ]:
n_qubits = 8
depth = 3
circ_ansatz02 = ansatz_qlm_02(nqubits=n_qubits, depth=depth)

In [ ]:
%qatdisplay circ_ansatz02 --svg

In [ ]:
filename = "ansatz_{}_depth_{}_nqubits_{}".format("simple02", depth, n_qubits)
# Create the parameter values
parameters = {v_ : 2 * np.pi * np.random.rand() for i_, v_ in enumerate(circ_ansatz02.get_variables())}
angles = [k for k, v in parameters.items()]
values = [v for k, v in parameters.items()]
pdf_parameters = pd.DataFrame([angles, values], index=['key', 'value']).T

In [ ]:
circ_ansatz02_, _ = angles_ansatz01(circ_ansatz02, pdf_parameters)

In [ ]:
class_dict = {
    'nqubits' : n_qubits,    
    "qpu" : qpu,
    "parameters" : pdf_parameters,
    "filename": folder + filename,
    "save": True    
}
solv_ansatz02 = SolveCircuit(circ_ansatz02_, **class_dict)
solv_ansatz02.run()

In [ ]:
solv_ansatz02.state

### 1.3 Other ansatzes

We can solve different ansatzes with the *SolveCircuit* the only mandatory input is the myqlm circuit of the ansatz with the parameters set.

In [ ]:
#Ansatzes built in the myqlm atos library
from qat.fermion.circuits import make_ldca_circ, make_general_hwe_circ

In [ ]:
nqubit = 8
depth = 3
lda_circ = make_ldca_circ(nqubit, depth)

In [ ]:
%qatdisplay lda_circ --svg

In [ ]:
filename = "ansatz_{}_depth_{}_nqubits_{}".format("ldca", depth, nqubit)
# Create the parameter values
parameters = {v_ : 2 * np.pi * np.random.rand() for i_, v_ in enumerate(lda_circ.get_variables())}
angles = [k for k, v in parameters.items()]
values = [v for k, v in parameters.items()]
pdf_parameters = pd.DataFrame([angles, values], index=['key', 'value']).T

In [ ]:
lda_circ_, _ = angles_ansatz01(lda_circ, pdf_parameters)

In [ ]:
%qatdisplay lda_circ_ --svg

In [ ]:
class_dict = {
    'nqubits' : nqubit,    
    "qpu" : qpu,
    "parameters" : pdf_parameters,
    "filename": folder + filename,
    "save": True    
}
solv_ldca = SolveCircuit(lda_circ_, **class_dict)
solv_ldca.run()

In [ ]:
solv_ldca.state

In [ ]:
hwe_circ = make_general_hwe_circ(nqubit, n_cycles=1)

In [ ]:
%qatdisplay hwe_circ --svg

In [ ]:
filename = "ansatz_{}_depth_{}_nqubits_{}".format("hwe", 1, nqubit)
# Create the parameter values
parameters = {v_ : 2 * np.pi * np.random.rand() for i_, v_ in enumerate(hwe_circ.get_variables())}
angles = [k for k, v in parameters.items()]
values = [v for k, v in parameters.items()]
pdf_parameters = pd.DataFrame([angles, values], index=['key', 'value']).T
hwe_circ_, _ = angles_ansatz01(hwe_circ, pdf_parameters)

In [ ]:
class_dict = {
    'nqubits' : nqubit,        
    "qpu" : qpu,
    "parameters" : pdf_parameters,
    "filename": folder + filename,
    "save": True    
}
solv_hwe = SolveCircuit(hwe_circ_, **class_dict)
solv_hwe.run()

In [ ]:
%qatdisplay hwe_circ_ --svg

In [ ]:
solv_hwe.state

### 1.4 Ansatz selector

In order to simplify the ansatz selection a function called **ansatz_selector** was built. 

In [ ]:
from PH.ansatzes.ansatzes import ansatz_selector

In [ ]:
conf_dict = {
    'nqubits' : 4,
    'depth' : 20
}

In [ ]:
circuit = ansatz_selector('simple01', **conf_dict)
%qatdisplay circuit --svg

In [ ]:
parameters = {v_ : 2 * np.pi * np.random.rand() for i_, v_ in enumerate(circuit.get_variables())}

In [ ]:
circuit = circuit(**parameters)

In [ ]:
%qatdisplay circuit --svg

In [ ]:
circuit = ansatz_selector('simple02', **conf_dict)
%qatdisplay circuit --svg

In [ ]:
circuit = ansatz_selector('lda', **conf_dict)
%qatdisplay circuit --svg

In [ ]:
circuit = ansatz_selector('hwe', **conf_dict)
%qatdisplay circuit --svg

## 1.5 run_ansatz

The complete ansatz-solving process can be done by using the function *run_ansatz* of the **ansatzes** module. A complete configuration dictionary should be provided and the function executes the complete steps for ansatz creation and solve.

In [ ]:
from PH.ansatzes.ansatzes import run_ansatz

In [ ]:
configuration ={
    "nqubits": 4,
    "depth": 2,
    "ansatz" : "simple01",
    "qpu": qpu,
    "t_inv": True,
    "folder": folder,
    "save": True,
    "solve" : True,
}

In [ ]:
output_dict = run_ansatz(**configuration)

In [ ]:
# State of the ansatz
output_dict["state"]

In [ ]:
# Parameters of the ansatz
output_dict["parameters"]

In [ ]:
# Base file name for the created files
output_dict["filename"]

The results will be stored under the selected folder (kwarg **folder**). The following files are created inside it:

* *_parameters.csv*: with the parameters of the ansatz 
* *_solve_ansatz_time.csv*: with the times of solving the ansatz
* *_state.csv*: complete state of the ansatz.

The base name for all the files will be: **ansatz\_{}\_nqubits\_{}\_depth\_{}\_qpu_ansatz\_{}**.


## 2. Line command

Additionally, the **ansatzes** module can be used from the command line and several parameters can be provided for configuring the ansatz. 

    For getting a help type:  *python ansatzes.py -h*.

    usage: ansatzes.py [-h] [-nqubits NQUBITS] [-depth DEPTH] [-ansatz ANSATZ] [-qpu_ansatz QPU_ANSATZ] [-folder FOLDER] [-filename FILENAME] [--save]
                       [--solve | --submit | --get_job] [-jobid JOB_ID]

    optional arguments:
      -h, --help            show this help message and exit
      -nqubits NQUBITS      Number of qbits for the ansatz.
      -depth DEPTH          Depth for ansatz.
      -ansatz ANSATZ        Ansatz type: simple01, simple02, lda or hwe.
      -qpu_ansatz QPU_ANSATZ
                            QPU for ansatz simulation:[qlmass_linalg, qlmass_mps, python, c, mps, linalg]
      -folder FOLDER        Path for storing results
      -filename FILENAME    Base Filename for saving. Only Valid with get_job
      --save                For storing results
      --solve               For solving complete ansatz
      --submit              For submiting ansatz to QLM
      --get_job             For getting a job from QLM
      -jobid JOB_ID         jobid of the QLM job


When **--save** is provided the results will be stored under the selected **FOLDER** (-folder FOLDER). The following files will be created inside it:

* *_parameters.csv*: with the parameters of the ansatz 
* *_solve_ansatz_time.csv*: with the times of solving the ansatz
* *_state.csv*: complete state of the ansatz.


**Example**

The following command will execute an ansatz computation for the simple ansatz with random angles for all the gates (*simple_02* ansatz) of 6 **qubits**, with 2 layers (**depth**) using the **c lineal algebra myqlm QPU** and the files will be saved into the folder **Saves**:

    python ansatzes.py -nqubits 6 -ansatz simple02 -depth 2 -qpu_ansatz c -folder ../Saves/ --solve --save

In the *../Saves/* folder, you should find the following files:

* ansatz_simple02_nqubits_6_depth_2_qpu_ansatz_CLinalg_parameters.csv
* ansatz_simple02_nqubits_6_depth_2_qpu_ansatz_CLinalg_solve_ansatz_time.csv
* ansatz_simple02_nqubits_6_depth_2_qpu_ansatz_CLinalg_state.csv


## 3. Submitting jobs to QLM (only for QLM users)

If the ansatz has a great number of qubits or has high depths computations using **myQLM** can not be done (or need a lot of time). In these cases, computations can be executed in an **Eviden QLM hardware platform** (CESGA has one available for their users able to simulate until 35 qubits). In this case, the computation can be submitted to the **QLM** and retrieved when finished.
For submitting an ansatz computation to the **QLM**, in addition to the arguments shown before, the following argument should be provided:

* --submit: For submitting ansatz to QLM

This option is valid only when **QPU_ANSATZ** (*-qpu_ansatz QPU_ANSATZ*) is: **qlmass** or **mps**. The execution will create the ansatz and submit the computation to QLM returning a *jobid*. **PLEASE keep this jobid for accessing the job.**

Once the computation is submitted to QLM we need to retrieve the results from the **QLM**. The following arguments can be used for getting the state:

* --get_job: For getting a job from QLM
* -jobid JOB_ID: jobid of the QLM job. Only when provided --get_job
* -filename FILENAME: Base Filename for saving. Only Valid with get_job
* --save: For storing results

If **--save** is provided the *-filename* should be provided too. The state will be saved as a csv with the name: **FILENAME_state.csv**

## 4. Masive ansatzes computations

For sending several ansatzes at the same time the following files can be used:

* **ansatzes.json**: JSON file with the configuration of the desired ansatzes. Each possible configuration parameter in the JSON file has associated a list with the desired values. The total number of ansatzes will be all the possible combinations of the values of all the parameters:
    
    Example of ansatzes.json:
    
        [
        {
            "nqubits": [12, 14],
            "depth": [1, 2, 3, 4],
            "ansatz" : ["simple01"],
            "qpu_ansatz": ["qlmass_linalg"],
            "t_inv": [true],
            "folder":  ["Test"],
            "save": [true],
            "solve" : [false],
            "submit": [true]
        }
    ]


    In this case, we ask for 2 ansatzes of 12 and 14 qubits and for each one we want 4 different depths (1, 2, 3 and 4). So with this JSON we are going to create 2 * 4 = 8 ansates circuits.
    
* **launch_ansatzes.py**: this script allows to configuration of the ansatzes taking the configuration from the **ansatzes.json** and executing them. To get help use:




python launch_ansatzes.py -h

    usage: launch_ansatzes.py [-h] [--all | -id ID] [--print] [--exe] [--count]

    optional arguments:
      -h, --help  show this help message and exit
      --all       For executing or submitting all anstazes from ansatzes.json
      -id ID      Select one element of all the anstazes from ansatzes.json
      --print     For printing the ansatz configuration.
      --exe       For executing program
      --count     Getting the number of ansatzes from ansatzes.json

## 5. Massive retrieving QLM

When using the **launch_ansatzes.py** you can send all the computations to the QLM (**submit** to True in the **ansatzes.json**). You can retrieve all the jobs with their corresponding JobId. You can use the  **launch\_get\_jobs.py** script to recover all the jobs automatically. This file uses the JSON file **get\_jobs.json**. In the JSON file, the JobIDs should be provided in the **job\_id** field of the JSON. In this case, one dictionary for JobID should be configured in the JSON file.

To get the help use:

    pyhton launch_get_jobs.py
    
    usage: launch_get_jobs.py [-h] [-filelist FILELIST] [--all | -id ID] [--print] [--exe] [--count]

    optional arguments:
      -h, --help          show this help message and exit
      -filelist FILELIST  Filename with folder to use
      --all               Select all the jobs from ansatzes.json
      -id ID              Select one element from ansatzes.json
      --print             For printing the configuration.
      --exe               For executing program
      --count             Getting the number of elements

